In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [3]:
customer = "ESN"
source = "facebook"
url = return_api_url()
url = "https://api.innkeepr.ai/api"
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [4]:
sources = call_api_with_accountId(
    f"{url}/sources/query",
    account_id,
    {"name":source},
    logging
)
print(sources)
if len(sources) != 1:
    raise Exception(f"more or less than one source was found: {sources}")
source_id = sources[0]["id"]
print(f"source_id = {source_id}")

In [5]:
treatments = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"source":source_id},
    logging
)
treatments = pd.json_normalize(treatments)

In [6]:
treatments["source"].value_counts()

In [7]:
treatments

In [10]:
treatments["relates_to.campaign.name"].value_counts()

In [11]:
if 'relates_to.adset.name' in treatments.columns:
    vc = treatments.groupby(by=["relates_to.campaign.name","relates_to.adset.name","properties.adAccountName"])['id'].value_counts()
elif "relates_to.assetGroup.name" in treatments.columns:
    vc = treatments.groupby(by=["relates_to.campaign.name","relates_to.assetGroup.name","properties.adAccountName"])['id'].value_counts()
else:
    vc = treatments.groupby(by=["relates_to.campaign.name","properties.adAccountName"])['id'].value_counts()
vc = vc.reset_index()
vc

In [23]:
vc_filtered = vc[
    (
        vc["relates_to.adset.name"].str.contains("Isoclear", case=False) #|
        #vc["relates_to.campaign.name"].str.contains("Öl", case=False) |
        #vc["relates_to.campaign.name"].str.contains("Oel", case=False) 
    )
    ]
vc_filtered

In [24]:
vc_filtered["relates_to.adset.name"].unique()

In [25]:
treatments_filtered=treatments[treatments["id"].isin(vc_filtered["id"].unique().tolist())]
treatments_filtered#["properties.status"].value_counts()

In [26]:
treatments_list = vc_filtered["id"].unique().tolist()
len(treatments_list), treatments_list

In [ ]:
test = send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"externalId":treatments_filtered["externalId"].dropna().unique().tolist()},
    logging
)
test

In [ ]:
ids_to_check = [
    "689d2ee3d3c5574d637e2f38",
    "689d2ee3d3c5574d6d51ba72",
    "689d2ee3d3c5574da13cc04f",
    "689d2ee3d3c5579d4248c90b",
    "689d2ee3d3c557ab891e2681",
    "689d2ee3d3c557ab8a3b7288",
    "689d2ee3d3c557ab8d3480f6",
    "689d2ee3d3c557ab8e18fc21",
    "689d2ee3d3c557ab977c2ea8",
    "689d2ee3d3c557c69c701a23",
    "689d2ee3d3c557c79d369ca6",
    "689d2ee4d3c5570ed570909f",
    "689d2ee4d3c5574d4b1b0f37",
    "689d2ee4d3c5574da916b78d",
    "689d2ee4d3c5574daa60a258",
    "689d2ee4d3c55764a55ceea8",
    "689d2ee4d3c55764a9716424",
    "689d2ee4d3c557761b6d8b48",
    "689d2ee4d3c55776e41c65e1",
    "689d2ee4d3c557bee62f6c6c",
    "689d2ee4d3c557c6b15f0f97",
    "689e8159d3c5576ee2309739",
    "68ae532dd3c5571934415f58",
]
treatments_views = send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"id":ids_to_check},
    logging
)
treatments_views = pd.json_normalize(treatments_views)

In [ ]:
treatments_views

In [ ]:
signals = send_to_innkeepr_api_paginated(
    f"{url}/signals/query",
    account_id,
    {"externalId":['123', '1435494216', '267300181', '752742929']},
    logging
)
signals